In [1]:
import LinearAlgebra: norm
import Statistics as st
using DelimitedFiles
using Healpix
using Plots

In [ ]:
function bias(a,b)
        return norm(a+b)/norm(a)
end
function s2c(r,alpha, delta)
    alpha,delta = deg2rad(alpha),deg2rad(delta)
    x = r*cos(alpha) * cos(delta)
    y = r*cos(delta) * sin(alpha)
    z = r*sin(delta)
    return [x, y, z]
end
r,a,d = 0.007,167,-7
t = s2c(r,a,d)
arr = []
while length(arr)<1500
    push!(arr,t)
end
b,err,w,e = [],[],[],[]
for i in ["output.dat","outputgal.dat","outputsgal.dat"]
    a = (readdlm(i), :auto)[1]
    s,m,dx,dy,dz = a[:,1],a[:,2],a[:,3],a[:,4] ,a[:,5]
    v,x = [],[]
    for i in 1:length(a[:,1])
        push!(x,[(dx[i]/m[i]),(dy[i]/m[i]),(dz[i]/m[i])])
    end
    v = bias.(arr,x)
    q = sqrt(length(s)/length(unique(s)))
    push!(b,[st.mean(v[1:300]),st.mean(v[301:600]),st.mean(v[601:900]),st.mean(v[901:1200]).,st.mean(v[1201:1500])])
    push!(err,[st.std(v[1:300])/q,st.std(v[301:600])/q,st.std(v[601:900])/q,st.std(v[901:1200])/q,,st.std(v[1201:1500])/q])
    push!(w,unique(s))
end
plot(w[1],b[1],seriestype = :scatter,xaxis=:log,grid=true,minorgrid=true,yerror=err[1],label="Complete Sky")
plot!(w[2],b[2],seriestype = :scatter,yerror=err[2],label="Galactic Cut (+-30 Degrees)")
plot!(w[3],b[3],seriestype = :scatter,yerror=err[3],label="Super-Galactic Cut (+-30 Degrees)")
xlabel!("Dataset Size")
ylabel!("Bias")
title!("Bias vs Souce Count (300 iterations for each point)")
#savefig("Bias-Net.png")